In [1]:
%run functions_15-05.ipynb

In [2]:
#Select significant columns    
significant_columns = [
"known_col_0",
"known_col_1",
"known_col_3",
"known_col_4",]

In [3]:
def do_everything(dataset):
    #Data Preprocessing
    dfr_dev_with_label, dfr_test_with_label, a, r, r_dev, r_test = data_preprocessing(dataset, "is_accepted", "y", 0.8)
    
    #Create rej datase with and without id
    r_dev_mod, r_test_mod = select_columns_rejects_without_id(
        r_dev, r_test, "r_dev_mod", "r_test_mod"
    )
    r_dev_mod_id, r_test_mod_id = select_columns_rejects_with_id(
        r_dev, r_test, "r_dev_mod_id", "r_test_mod_id"
    )
    
    #Create X and y
    X_res, y_res = create_X_y(a)
    
    #Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=7)
    columns = X_train.columns
    
    #50% of the accepted data
    n = int(0.5*(len(X_train)))
    X_train = X_train[:n]
    y_train = y_train[:n]
    
    conservative_dr = (
    1.1
    * len(y_test[y_test["target"] == 1])
    / (len(y_test[y_test["target"] == 1]) + len(y_test[y_test["target"] == 0])))

    # Columns
    X_train = pd.DataFrame(data=X_train, columns=columns)
    y_train = pd.DataFrame(data=y_train, columns=["target"])
    
    X_train = X_train[significant_columns]
    X_test_3 = X_test[significant_columns]
    r_dev_mod = r_dev_mod[significant_columns]
    r_test_mod = r_test_mod[significant_columns]
        
    #Rejected Test Sample
    dfr_dev_with_label.rename(columns={"y": "target"}, inplace=True)
    dfr_dev_with_label_X = dfr_dev_with_label[significant_columns]
    dfr_dev_with_label_y = dfr_dev_with_label[["target"]]
    dfr_test_with_label.rename(columns={"y": "target"}, inplace=True)
    dfr_test_with_label_X = dfr_test_with_label[significant_columns]
    dfr_test_with_label_y = dfr_test_with_label[["target"]]
    
    #Logistic regression    
    logreg, y_pred = log_reg(X_train, y_train, X_test_3)
    
    # Predictions on Accepted Test Sample
    pred_test1 = pred(y_test, X_test, X_test_3, logreg)

    # Predictions on Rejected Test Sample
    pred_dev_rej = pred(
        dfr_dev_with_label_y, dfr_dev_with_label, dfr_dev_with_label_X, logreg
    )
    pred_test_rej = pred(
        dfr_test_with_label_y, dfr_test_with_label, dfr_test_with_label_X, logreg
    )
    pred_rej1 = pred(
        dfr_test_with_label_y, dfr_test_with_label, dfr_test_with_label_X, logreg
    )
    
        # Prediction before RI
    ri1_train_rej = pred_dev_rej[["target_x", "id", "prediction_beforeRI"]]
    ri1_train_rej["prediction_beforeRI"] = ri1_train_rej["prediction_beforeRI"].apply(
        lambda x: 0
        if (x < ri1_train_rej["prediction_beforeRI"].quantile(q=1 - conservative_dr))
        else 1
    )
    ri1_test_rej = pred_test_rej[["target_x", "id", "prediction_beforeRI"]]
    ri1_test_rej["prediction_beforeRI"] = ri1_test_rej["prediction_beforeRI"].apply(
        lambda x: 0
        if (x < ri1_test_rej["prediction_beforeRI"].quantile(q=1 - conservative_dr))
        else 1
    )
    pred_rej1 = pred_rej1[["target_x", "id", "prediction_beforeRI"]]
    pred_rej1["prediction_beforeRI"] = pred_rej1["prediction_beforeRI"].apply(
        lambda x: 0
        if (x < pred_rej1["prediction_beforeRI"].quantile(q=1 - conservative_dr))
        else 1
    )
    ri1_train_rej.rename(columns={"target_x": "target"}, inplace=True)
    ri1_test_rej.rename(columns={"target_x": "target"}, inplace=True)
    pred_rej1.rename(columns={"target_x": "target"}, inplace=True)
    
    #Isolation forest    
    r_dev_mod, r_test_mod = isolation_forest(X_train, r_dev_mod, r_test_mod)
    
    return dfr_dev_with_label, dfr_test_with_label, dfr_dev_with_label_y, dfr_dev_with_label_X, dfr_test_with_label_X, dfr_test_with_label_y, r_dev_mod, r_test_mod, r_dev_mod_id, r_test_mod_id, X_train, X_test, X_test_3, y_train, y_test, pred_test1, ri1_train_rej, ri1_test_rej,pred_rej1, logreg

## Semi-Supervised Learning

In [4]:
def semi_supervised_acc(logreg):
    #Print AUC score of logreg
    logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test_3))
    # Train
    train_new_model = ssl_prep(
    X_train,
    y_train,
    r_dev_mod,
    )
    # Test
    test_new_model = ssl_prep(
    X_test_3,
    y_test,
    r_test_mod,
    )
    X_ssl, y_ssl = ssl_split(train_new_model, "unlabel")
    
    outcome_a["Flag"] = outcome_a.apply(flag_df_beforeRI, axis=1)
    kickout_beforeRI_acc = kickout_beforeRI(outcome_a)
    outcome_a["Flag"] = outcome_a.apply(flag_df_baseline, axis=1)
    kcikout_baseline_acc = kickout_baseline(outcome_a)
    
    ############# SEMI-SUPERVISED: SELF-TRAINING & LABEL PROPAGATION ############
    
    #SELF-TRAINING
    self_training = ssl_model_selftraining(X_ssl, y_ssl, SelfTrainingClassifier)
    x_st = ssl_predictions_oth(self_training, X_test_3)
    
    x_st["Flag"] = x_st.apply(flag_df_ssl, axis=1)
    kickout_st_acc = kickout_ssl(x_st)
 
    #LABEL PROPAGATION
    label_propagation = ssl_model_label(X_ssl, y_ssl, LabelPropagation)
    x_lp = ssl_predictions_oth(label_propagation, X_test_3)
    
    x_lp["Flag"] = x_lp.apply(flag_df_ssl, axis=1)
    kickout_lp_acc = kickout_ssl(x_lp)
    
    #LABEL SPREADING
    label_spreading = ssl_model_label(X_ssl, y_ssl, LabelSpreading)
    x_sp = ssl_predictions_oth(label_spreading, X_test_3)
    
    x_sp["Flag"] = x_sp.apply(flag_df_ssl, axis=1)
    kickout_ls_acc = kickout_ssl(x_sp)
    

    ################ SEMI-SUPERVISED: ACTIVE LEARNING #############
    
#     #WITH 20% OF THE REJECTED DATA
#     regressor_20 = active_learning2(X_train, y_train, r_dev_mod, 0.2)
#     x_al_20 = ssl_predictions_oth(regressor_20, X_test_3)
#     x_al_20["Flag"] = x_al_20.apply(flag_df_ssl, axis=1)
#     kickout_al_20 = kickout_ssl(x_al_20)
    
    ##WITH SELF-TRAINING CLASSIFIER AND 50% OF THE REJECTED DATA
    
#     regressor_50 = active_learning3(X_train, y_train, r_dev_mod, 0.5)
#     x_al_50 = ssl_predictions_oth2(regressor_50, X_test_3)
#     x_al_50["Flag"] = x_al_50.apply(flag_df, axis=1)
#     kickout_al_50_st = kickout(x_al_50)
    
#     #WITH 50% OF THE REJECTED DATA
#     regressor_50 = active_learning2(X_train, y_train, r_dev_mod, 0.2)
#     x_al_50 = ssl_predictions_oth2(regressor_50, X_test_3)
#     x_al_50["Flag"] = x_al_50.apply(flag_df, axis=1)
#     kickout_al_50 = kickout(x_al_50)
 
    ############# SELECT ESTIMATORS WITH THE BEST RESULTS ############
#     estimators = []
#     if kickout_lp > kickout_st and kickout_lp > kickout_ls:
#         estimators.append(("lp", label_propagation))
#         max_ssl_kickout = kickout_lp
#     elif kickout_st > kickout_ls and kickout_st > kickout_lp:
#         estimators.append(("st", self_training))
#         max_ssl_kickout = kickout_st
#     elif kickout_ls > kickout_st and kickout_ls > kickout_lp:
#         estimators.append(("ls", label_spreading))
#         max_ssl_kickout = kickout_ls
    
                          
#     if kickout_al_20 > kickout_al_50:
#         estimators.append(("al_20", regressor_20))
#         max_al_kickout = kickout_al_20
#     else:
#         estimators.append(("al_50", regressor_50))
#         max_al_kickout = kickout_al_50
                  
    ############# ENSEMBLING ############
    
    #x_ds = ssl_predictions_ds(X_test_3, estimators)
    #x_ds["Flag"] = x_ds.apply(flag_df, axis=1)
    #kickout_ds = kickout(x_ds)
    return logit_roc_auc, kickout_beforeRI_acc, kcikout_baseline_acc, kickout_st_acc, kickout_lp_acc, kickout_ls_acc

In [5]:
def semi_supervised_rej(logreg):
    #Print AUC score of logreg
    logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test_3))
    # Train
    train_new_model = ssl_prep(
    X_train,
    y_train,
    r_dev_mod,
    )
    # Test
    test_new_model = ssl_prep(
    X_test_3,
    y_test,
    r_test_mod,
    )
    X_ssl, y_ssl = ssl_split(train_new_model, "unlabel")
    
    outcome_b["Flag"] = outcome_b.apply(flag_df_beforeRI, axis=1)
    kickout_beforeRI_rej = kickout_beforeRI(outcome_b)
    outcome_b["Flag"] = outcome_b.apply(flag_df_baseline, axis=1)
    kcikout_baseline_rej = kickout_baseline(outcome_b)
    
    ############# SEMI-SUPERVISED: SELF-TRAINING & LABEL PROPAGATION ############
    
    #SELF-TRAINING
    self_training = ssl_model_selftraining(X_ssl, y_ssl, SelfTrainingClassifier)
    x_rej_st = ssl_predictions_oth_rej(self_training, dfr_test_with_label_X)
    
    x_rej_st["Flag"] = x_rej_st.apply(flag_df_ssl, axis=1)
    kickout_st_rej = kickout_ssl(x_rej_st)
 
    #LABEL PROPAGATION
    label_propagation = ssl_model_label(X_ssl, y_ssl, LabelPropagation)
    x_rej_lp = ssl_predictions_oth_rej(label_propagation, dfr_test_with_label_X)
    
    x_rej_lp["Flag"] = x_rej_lp.apply(flag_df_ssl, axis=1)
    kickout_lp_rej = kickout_ssl(x_rej_lp)
    
    #LABEL SPREADING
    label_spreading = ssl_model_label(X_ssl, y_ssl, LabelSpreading)
    x_rej_ls = ssl_predictions_oth_rej(label_spreading, dfr_test_with_label_X)
    
    x_rej_ls["Flag"] = x_rej_ls.apply(flag_df_ssl, axis=1)
    kickout_ls_rej = kickout_ssl(x_rej_ls)
    

    ################ SEMI-SUPERVISED: ACTIVE LEARNING #############
    
#     #WITH 20% OF THE REJECTED DATA
#     regressor_20 = active_learning2(X_train, y_train, r_dev_mod, 0.2)
#     x_al_20 = ssl_predictions_oth(regressor_20, X_test_3)
#     x_al_20["Flag"] = x_al_20.apply(flag_df_ssl, axis=1)
#     kickout_al_rej = kickout_ssl(x_al_20)
    
    ##WITH SELF-TRAINING CLASSIFIER AND 50% OF THE REJECTED DATA
    
#     regressor_50 = active_learning3(X_train, y_train, r_dev_mod, 0.5)
#     x_al_50 = ssl_predictions_oth2(regressor_50, X_test_3)
#     x_al_50["Flag"] = x_al_50.apply(flag_df, axis=1)
#     kickout_al_50_st = kickout(x_al_50)
    
#     #WITH 50% OF THE REJECTED DATA
#     regressor_50 = active_learning2(X_train, y_train, r_dev_mod, 0.2)
#     x_al_50 = ssl_predictions_oth2(regressor_50, X_test_3)
#     x_al_50["Flag"] = x_al_50.apply(flag_df, axis=1)
#     kickout_al_50 = kickout(x_al_50)
 
    ############# SELECT ESTIMATORS WITH THE BEST RESULTS ############
#     estimators = []
#     if kickout_lp > kickout_st and kickout_lp > kickout_ls:
#         estimators.append(("lp", label_propagation))
#         max_ssl_kickout = kickout_lp
#     elif kickout_st > kickout_ls and kickout_st > kickout_lp:
#         estimators.append(("st", self_training))
#         max_ssl_kickout = kickout_st
#     elif kickout_ls > kickout_st and kickout_ls > kickout_lp:
#         estimators.append(("ls", label_spreading))
#         max_ssl_kickout = kickout_ls
    
                          
#     if kickout_al_20 > kickout_al_50:
#         estimators.append(("al_20", regressor_20))
#         max_al_kickout = kickout_al_20
#     else:
#         estimators.append(("al_50", regressor_50))
#         max_al_kickout = kickout_al_50
                  
    ############# ENSEMBLING ############
    
    #x_ds = ssl_predictions_ds(X_test_3, estimators)
    #x_ds["Flag"] = x_ds.apply(flag_df, axis=1)
    #kickout_ds = kickout(x_ds)
    return logit_roc_auc, kickout_beforeRI_rej, kcikout_baseline_rej, kickout_st_rej, kickout_lp_rej, kickout_ls_rej

## Run for all datasets

In [6]:
a_directory = "C:/Users/Asus/Desktop/Repo/MasterThesis_RI/Data_09_05/"

my_dataframes_acc = []
my_dataframes_rej = []
for filename in os.listdir(a_directory):
    filepath = os.path.join(a_directory)
    dfr_dev_with_label, dfr_test_with_label, dfr_dev_with_label_y, dfr_dev_with_label_X, dfr_test_with_label_X, dfr_test_with_label_y, r_dev_mod, r_test_mod, r_dev_mod_id, r_test_mod_id, X_train, X_test, X_test_3, y_train, y_test, pred_test1, ri1_train_rej, ri1_test_rej,pred_rej1, logreg = do_everything(filename)
    #Calculate thresholds
    conservative_dr = (
    1.1
    * len(y_test[y_test["target"] == 1])
    / (len(y_test[y_test["target"] == 1]) + len(y_test[y_test["target"] == 0])))
    #Ri1_train    
    ri1_train = predictions1(logreg)
    #ri1_train = predictions1(logreg, r_dev_mod, r_dev_mod_id)
    #evaluation(ri1_train, X_test)
    outcome_a = evaluation(ri1_train)
    outcome_b = evaluation_rejects(ri1_train_rej)  
    #Calculate kickout measures Accepts
    kickout_list_acc = list(semi_supervised_acc(logreg))
    new_filename_acc = filename[:-4]
    new_filename2_acc = pd.DataFrame(kickout_list_acc).transpose()
    new_filename2_acc = new_filename2_acc.rename(columns={0: 'logit_roc_auc', 1: 'kickout_beforeRI_acc', 2: 'kcikout_baseline_acc', 3: 'kickout_st_acc', 4:'kickout_lp_acc', 5:'kickout_ls_acc'}, index={0: new_filename_acc})
    my_dataframes_acc.append(new_filename2_acc)
    #Calculate kickout measures Rejects
    kickout_list_rej = list(semi_supervised_rej(logreg))
    new_filename_rej = filename[:-4]
    new_filename2_rej = pd.DataFrame(kickout_list_rej).transpose()
    new_filename2_rej = new_filename2_rej.rename(columns={0: 'logit_roc_auc', 1: 'kickout_beforeRI_rej', 2: 'kcikout_baseline_rej', 3: 'kickout_st_rej',  4:'kickout_lp_rej', 5:'kickout_ls_rej'}, index={0: new_filename_rej})
    my_dataframes_rej.append(new_filename2_rej)
df_results_acc = pd.concat(my_dataframes_acc, axis = 0)
df_results_rej = pd.concat(my_dataframes_rej, axis = 0)

In [7]:
df_results_acc

,logit_roc_auc,kickout_beforeRI_acc,kcikout_baseline_acc,kickout_st_acc,kickout_lp_acc,kickout_ls_acc
cons_scen1_1,0.832434,0.955000,0.955000,0.952778,0.950556,0.951667
cons_scen1_2,0.697131,0.908889,0.915556,0.912222,0.893333,0.895556
cons_scen1_3,0.522209,0.876111,0.878333,0.871667,0.867222,0.868333
cons_scen2_1,0.895877,0.935000,0.936111,0.940556,0.927222,0.928333
cons_scen2_2,0.753397,0.858889,0.853333,0.854444,0.838889,0.842222
cons_scen2_3,0.546092,0.746667,0.743333,0.753333,0.740000,0.741111
cons_scen3_1,0.855507,0.958750,0.960000,0.957500,0.951250,0.953750
cons_scen3_2,0.698942,0.912500,0.915000,0.922500,0.903750,0.906250
cons_scen3_3,0.559711,0.862500,0.867500,0.862500,0.861250,0.861250
cons_scen4_1,0.910588,0.940000,0.940000,0.938750,0.937500,0.938750


In [8]:
df_results_rej

,logit_roc_auc,kickout_beforeRI_rej,kcikout_baseline_rej,kickout_st_rej,kickout_lp_rej,kickout_ls_rej
cons_scen1_1,0.832434,0.885000,0.885000,0.885000,0.865000,0.875000
cons_scen1_2,0.697131,0.840000,0.840000,0.840000,0.790000,0.800000
cons_scen1_3,0.522209,0.825000,0.825000,0.805000,0.805000,0.795000
cons_scen2_1,0.895877,0.770000,0.770000,0.760000,0.760000,0.760000
cons_scen2_2,0.753397,0.775000,0.765000,0.765000,0.735000,0.735000
cons_scen2_3,0.546092,0.620000,0.620000,0.600000,0.630000,0.630000
cons_scen3_1,0.855507,0.905000,0.910000,0.905000,0.880000,0.875000
cons_scen3_2,0.698942,0.852500,0.862500,0.852500,0.832500,0.817500
cons_scen3_3,0.559711,0.790000,0.790000,0.780000,0.785000,0.805000
cons_scen4_1,0.910588,0.845000,0.845000,0.845000,0.820000,0.830000


### With 100% of the accepted data

In [ ]:
df_results_acc

In [ ]:
df_results_rej

### With 10% of the accepted data

In [ ]:
df_results_acc

In [ ]:
df_results_rej

### With 50% of the accepted data

In [ ]:
df_results_acc

In [ ]:
df_results_rej